# Json import to openSearch

In [1]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

# Login to openSearch

In [2]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]


----------------------------------------------------------------------------------- INDEX SETTINGS
{'user209': {'settings': {'index': {'creation_date': '1649108445739',
                                    'knn': 'true',
                                    'number_of_replicas': '0',
                                    'number_of_shards': '4',
                                    'provided_name': 'user209',
                                    'refresh_interval': '1s',
                                    'uuid': 'Abp6iFWNQ4qYsNVk4YMxqQ',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user209': {'mappings': {'properties': {'description': {'similarity': 'BM25',
                                                         'type': 'text'},
                                         'recipeId': {'type': 'keyword'},
                                         'sentence_embed

# Create index structure

In [3]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

# Choose the tokenizer and model that will be used in the embeddings

In [4]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [5]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json", tokenizer, model).createMap()

In [13]:
import indexRecipes as ir
import time

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()

#time.sleep(20)

updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated


In [15]:
import search as s

s.Search(client, index_name, tokenizer, model).queryOpenSearch('how to make chicken', 10)


Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': 'amzn1.alexa.kitchen.document.simplyrecipes.pdid.MTIwOTQ=',
                    '_index': 'user209',
                    '_score': 5.4563246,
                    '_source': {},
                    '_type': '_doc',
                    'fields': {'description': ['skinless boneless chicken '
                                               'breast marinate lime cilantro '
                                               'grill'],
                               'recipeId': ['amzn1.alexa.kitchen.document.simplyrecipes.pdid.MTIwOTQ='],
                               'title': ['Grilled Cilantro Lime Chicken']}},
                   {'_id': 'amzn1.alexa.kitchen.document.thekitchn.pdid.ODMxOQ==',
                    '_index': 'user209',
                    '_score': 5.2428713,
                    '_source': {},
                    '_type': '_doc',
                    'fields': {'descript

In [11]:
adfasdfdafdsafd
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)

NameError: name 'adfasdfdafdsafd' is not defined